In [23]:
import pandas as pd
import glob
from pathlib import Path
import pandas_profiling
from matplotlib import pyplot
import seaborn as sns
from pandas.plotting import autocorrelation_plot
from datetime import datetime

In [24]:
pd.set_option('display.max_rows', 1000)

In [25]:
all_filenames = glob.glob('/Users/ioneuk/Documents/flight-price-predictor/data/sky-scanner/**/*.csv', recursive=True)

In [26]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv( "/Users/ioneuk/Documents/flight-price-predictor/data/aggregate.csv", index=False, encoding='utf-8-sig')

In [27]:
df = pd.read_csv('/Users/ioneuk/Documents/flight-price-predictor/data/aggregate.csv')

In [28]:
df.head()

,date_time,departure_city,departure_iata_code,arrival_city,destination_iata_code,departure_date_time,arrival_date_time,flight_duration,carrier_name,agent_name,flight_number,price
0,2020-03-01 08:33:01.789164,Moscow,SVO,Budapest,BUD,2020-05-01T20:10:00,2020-05-01T21:55:00,165,Aeroflot,Aeroflot,2468,3710.37
1,2020-03-01 08:33:01.789164,Moscow,VKO,Budapest,BUD,2020-05-01T10:05:00,2020-05-01T11:40:00,155,Wizz Air,Wizz Air,2490,4763.70
2,2020-03-01 08:33:01.789164,Moscow,SVO,Budapest,WAW,2020-05-01T18:05:00,2020-05-01T19:15:00,130,LOT,LOT,5676,5306.59
3,2020-03-01 08:33:01.789164,Moscow,SVO,Budapest,BRU,2020-05-04T15:45:00,2020-05-04T18:25:00,220,Brussels Airlines,Brussels Airlines,2846,1879.37
4,2020-03-01 08:33:01.789164,Moscow,VKO,Budapest,BUD,2020-05-04T10:05:00,2020-05-04T11:40:00,155,Wizz Air,Wizz Air,2490,2177.55


In [29]:
df['measure_date'] = pd.to_datetime(pd.to_datetime(df['date_time']).dt.date)
df['departure_date'] = pd.to_datetime(pd.to_datetime(df['departure_date_time']).dt.date)
df['departure_hour'] = pd.to_datetime(df['departure_date_time']).dt.hour
df['departure_minute'] = pd.to_datetime(df['departure_date_time']).dt.minute
df['arrival_date'] = pd.to_datetime(pd.to_datetime(df['arrival_date_time']).dt.date)
df['arrival_hour'] = pd.to_datetime(df['arrival_date_time']).dt.hour
df['arrival_minute'] = pd.to_datetime(df['arrival_date_time']).dt.minute

In [30]:
df['days_left_to_departure'] = (df['departure_date']-df['measure_date']).transform(lambda x: x.dt.days)

In [31]:
df['departure_city'] = df['departure_city'].str.lower()
df['arrival_city'] = df['arrival_city'].str.lower()
df['carrier_name'] = df['carrier_name'].str.lower()
df['agent_name'] = df['agent_name'].str.lower()

In [32]:
del df['departure_iata_code']
del df['destination_iata_code']
del df['date_time']
del df['departure_date_time']
del df['arrival_date_time']

In [33]:
df.shape

(226391, 15)

In [34]:
df.head()

,departure_city,arrival_city,flight_duration,carrier_name,agent_name,flight_number,price,measure_date,departure_date,departure_hour,departure_minute,arrival_date,arrival_hour,arrival_minute,days_left_to_departure
0,moscow,budapest,165,aeroflot,aeroflot,2468,3710.37,2020-03-01,2020-05-01,20,10,2020-05-01,21,55,61
1,moscow,budapest,155,wizz air,wizz air,2490,4763.70,2020-03-01,2020-05-01,10,5,2020-05-01,11,40,61
2,moscow,budapest,130,lot,lot,5676,5306.59,2020-03-01,2020-05-01,18,5,2020-05-01,19,15,61
3,moscow,budapest,220,brussels airlines,brussels airlines,2846,1879.37,2020-03-01,2020-05-04,15,45,2020-05-04,18,25,64
4,moscow,budapest,155,wizz air,wizz air,2490,2177.55,2020-03-01,2020-05-04,10,5,2020-05-04,11,40,64


In [35]:
df.dtypes

departure_city                    object
arrival_city                      object
flight_duration                    int64
carrier_name                      object
agent_name                        object
flight_number                      int64
price                            float64
measure_date              datetime64[ns]
departure_date            datetime64[ns]
departure_hour                     int64
departure_minute                   int64
arrival_date              datetime64[ns]
arrival_hour                       int64
arrival_minute                     int64
days_left_to_departure             int64
dtype: object

In [36]:
df.drop_duplicates(['departure_city','arrival_city','flight_duration','carrier_name','agent_name','flight_number','measure_date','departure_date','departure_hour','departure_minute','arrival_date','arrival_hour','arrival_minute'],inplace = True)

In [37]:
df.shape

(219327, 15)

In [38]:
df['previous_price_for_trip'] = df.sort_values('measure_date').groupby(['departure_date','arrival_date','departure_city','arrival_city','flight_number','carrier_name'])['price'].shift(1).fillna(0)

In [39]:
df.head()

,departure_city,arrival_city,flight_duration,carrier_name,agent_name,flight_number,price,measure_date,departure_date,departure_hour,departure_minute,arrival_date,arrival_hour,arrival_minute,days_left_to_departure,previous_price_for_trip
0,moscow,budapest,165,aeroflot,aeroflot,2468,3710.37,2020-03-01,2020-05-01,20,10,2020-05-01,21,55,61,0.0
1,moscow,budapest,155,wizz air,wizz air,2490,4763.70,2020-03-01,2020-05-01,10,5,2020-05-01,11,40,61,0.0
2,moscow,budapest,130,lot,lot,5676,5306.59,2020-03-01,2020-05-01,18,5,2020-05-01,19,15,61,0.0
3,moscow,budapest,220,brussels airlines,brussels airlines,2846,1879.37,2020-03-01,2020-05-04,15,45,2020-05-04,18,25,64,0.0
4,moscow,budapest,155,wizz air,wizz air,2490,2177.55,2020-03-01,2020-05-04,10,5,2020-05-04,11,40,64,0.0


In [40]:
df[(df['departure_date']=='2020-03-12') & (df['departure_city']=='london')& (df['arrival_city']=='amsterdam') & (df['flight_number']==7405)].sort_values('measure_date')

,departure_city,arrival_city,flight_duration,carrier_name,agent_name,flight_number,price,measure_date,departure_date,departure_hour,departure_minute,arrival_date,arrival_hour,arrival_minute,days_left_to_departure,previous_price_for_trip
117440,london,amsterdam,55,easyjet,easyjet,7405,716.61,2020-02-04,2020-03-12,18,30,2020-03-12,20,25,37,0.00
32781,london,amsterdam,55,easyjet,easyjet,7405,733.62,2020-02-07,2020-03-12,18,30,2020-03-12,20,25,34,716.61
135474,london,amsterdam,55,easyjet,easyjet,7405,730.56,2020-02-10,2020-03-12,18,30,2020-03-12,20,25,31,733.62
73534,london,amsterdam,55,easyjet,easyjet,7405,696.76,2020-02-22,2020-03-12,18,30,2020-03-12,20,25,19,730.56
68236,london,amsterdam,55,easyjet,easyjet,7405,730.71,2020-02-25,2020-03-12,18,30,2020-03-12,20,25,16,696.76


In [41]:
df.to_csv('processed_dataset.csv', index=False)

In [42]:
df.dtypes

departure_city                     object
arrival_city                       object
flight_duration                     int64
carrier_name                       object
agent_name                         object
flight_number                       int64
price                             float64
measure_date               datetime64[ns]
departure_date             datetime64[ns]
departure_hour                      int64
departure_minute                    int64
arrival_date               datetime64[ns]
arrival_hour                        int64
arrival_minute                      int64
days_left_to_departure              int64
previous_price_for_trip           float64
dtype: object